# Test analysis

### Import libraries

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import netCDF4
# import pandas as pd
import dask
# from wrf import to_np
# from mpl_toolkits.basemap import Basemap
# from datetime import datetime
# from datetime import timedelta
# from scipy import stats
# import scipy.interpolate as interp
# from dask.distributed import Client
%matplotlib inline

In [3]:
# for quantreg
# from sklearn.linear_model import QuantileRegressor
from scipy import stats

In [4]:
from functions import ccplot, binning

In [5]:
import seaborn as sns
# Use seaborn style defaults and set default figure size
# plt.style.use('seaborn-pastel')
# sns.set_theme(style="ticks")
plt.style.use(['science', 'notebook'])

In [6]:
# Colormap selection
xr.set_options(cmap_divergent='RdBu', cmap_sequential='YlGnBu')

In [7]:
# client = Client(n_workers=30, threads_per_worker=1, memory_limit='60GB')

### Import data

In [8]:
mfdata_DIR1 = './data/GPM_lowres_data/gpm_200*.nc'
gpm = xr.open_mfdataset(mfdata_DIR1, engine='netcdf4', combine='nested', concat_dim='time', parallel=True)

In [9]:
# gpm = gpm.unify_chunks()
# gpm.chunks

In [10]:
gpm

<xarray.Dataset>
Dimensions:    (time: 168048, lon: 161, lat: 161)
Coordinates:
  * time       (time) datetime64[ns] 2000-06-01 ... 2009-12-31T23:30:00
  * lon        (lon) float64 60.0 60.25 60.5 60.75 ... 99.25 99.5 99.75 100.0
  * lat        (lat) float64 0.0 0.25 0.5 0.75 1.0 ... 39.25 39.5 39.75 40.0
Data variables:
    precipCal  (time, lat, lon) float32 dask.array<chunksize=(10272, 161, 161), meta=np.ndarray>

In [11]:
precip = gpm.precipCal.resample(time = '1H').mean()

In [12]:
# precip = precip.chunk(dict(time=5000, lat=40, lon=40))

In [13]:
# precip = precip.unify_chunks()
# precip.chunks

In [14]:
precip

<xarray.DataArray 'precipCal' (time: 84024, lat: 161, lon: 161)>
dask.array<stack, shape=(84024, 161, 161), dtype=float32, chunksize=(1, 161, 161), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 2000-06-01 ... 2009-12-31T23:00:00
  * lon      (lon) float64 60.0 60.25 60.5 60.75 61.0 ... 99.25 99.5 99.75 100.0
  * lat      (lat) float64 0.0 0.25 0.5 0.75 1.0 ... 39.0 39.25 39.5 39.75 40.0

In [15]:
mfdata_DIR2 = './data/era_pres_data/era_pres_200*.nc'
era = xr.open_mfdataset(mfdata_DIR2, engine='netcdf4', combine='nested', concat_dim='time', parallel=True)
# era = xr.open_mfdataset(mfdata_DIR2, engine='netcdf4', combine='nested', concat_dim='time', parallel=True)

In [16]:
with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    era = era.reindex(latitude = era.latitude[::-1])

In [17]:
# era = era.sel(time = slice("2000-06-01 00:00:00","2004-12-31 23:00:00"))

In [18]:
# era = era.unify_chunks()
# era.chunks

In [19]:
era

,Array,Chunk
Bytes,245.51 GiB,2.02 GiB
Shape,"(87672, 29, 161, 161)","(720, 29, 161, 161)"
Count,760 Tasks,190 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,245.51 GiB,2.02 GiB
Shape,"(87672, 29, 161, 161)","(720, 29, 161, 161)"
Count,760 Tasks,190 Chunks
Type,float32,numpy.ndarray
,Array,Chunk


In [20]:
# era = era.sel(expver=1, drop=True)

In [21]:
era = era.transpose('time', 'level', 'latitude', 'longitude')

In [22]:
# era = era.rename_dims({'longitude':'lon', 'latitude':'lat'})

In [23]:
era = era.rename({'longitude':'lon', 'latitude':'lat'})

In [24]:
era

,Array,Chunk
Bytes,245.51 GiB,2.02 GiB
Shape,"(87672, 29, 161, 161)","(720, 29, 161, 161)"
Count,760 Tasks,190 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,245.51 GiB,2.02 GiB
Shape,"(87672, 29, 161, 161)","(720, 29, 161, 161)"
Count,760 Tasks,190 Chunks
Type,float32,numpy.ndarray
,Array,Chunk


In [25]:
q = era.q
t = era.t
w = era.w
pres = era.coords['level']

In [26]:
p1 = precip.sel(lat = 19, lon=72, time=slice("2000-06-01 00:00:00","2002-09-30 23:00:00")).load()
w1 = w.sel(lat = 19, lon=72, time=slice("2000-06-01 00:00:00","2002-09-30 23:00:00")).load()
t1 = t.sel(lat = 19, lon=72, time=slice("2000-06-01 00:00:00","2002-09-30 23:00:00")).load()

In [29]:
p1

<xarray.DataArray 'precipCal' (time: 20448)>
array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 2000-06-01 ... 2002-09-30T23:00:00
    lon      float64 72.0
    lat      float64 19.0

In [41]:
# finding precipitation extremes

temporary_precip = []

precip95 = p1.where(p1 > 0.1).quantile(0.95)
for time in range(len(p1)):
    if p1[time] > precip95:
        temporary_precip.append(p1[time])

In [44]:
np.array(temporary_precip)

array([10.392342 , 16.125036 , 46.463863 , 42.228115 , 44.30081  ,
       29.489807 , 13.527607 ,  7.19404  ,  9.955382 , 15.157883 ,
        8.272744 , 12.322466 , 17.726788 , 19.939861 , 14.250515 ,
        8.009617 ,  9.886288 ,  9.337    , 20.175713 ,  7.9922695,
        8.358852 ,  9.620819 , 14.74831  , 21.723772 ,  7.5160666,
        8.953884 , 14.075197 , 16.40002  , 11.948797 ,  9.121326 ,
       10.539016 , 13.9616585,  8.532866 ,  7.315008 ,  7.0493183,
       15.602955 , 14.366441 ,  8.269313 ,  7.455923 ,  8.227329 ,
       27.481884 , 20.630165 ,  8.586729 ,  8.34266  , 10.141513 ,
        9.821833 ,  7.622194 ,  9.61058  , 11.292687 , 13.294573 ,
        8.365073 , 14.4693165, 15.806767 ,  7.3898835,  9.958394 ,
       19.069233 , 19.551193 , 13.953827 ,  9.541302 ,  7.988985 ,
        7.9409904,  7.681491 , 10.56789  ,  8.304456 ,  7.6548653,
        7.793007 ,  7.7750044, 15.138235 , 26.563324 , 25.316805 ,
       14.369713 , 11.141237 , 15.569366 , 23.955961 , 16.5469

In [26]:
# p1 = precip.sel(time = "2000-06-02 01:00:00")
# w1 = w.sel(time = "2000-06-02 01:00:00")
# t1 = t.sel(time = "2000-06-02 01:00:00")
# q1 = q.sel(time = "2000-06-02 01:00:00")

In [53]:
def get_qs(temp, pres):
    """
    Elemental Function to determine the precipitation estimate for each value of tempreature and omega at each grid point. (func to be used in starmap multithreading)
    ------------------
    Input :
        temp : temperature value at a grid point.
        omega_da : vertical velocity value at a grid point
    Output :
        qs : qs value at that grid point
    ------------------
    """
    a1 = 6.1114
    temp0 = 273.16
    a3w = 17.269
    a4w = 35.86
    a3i = 21.875
    a4i = 7.66

    # calculating saturation vapor pressure using temperature values
    if temp > temp0:
        a3 = a3w
        a4 = a4w
        es = a1 * np.exp(a3 * ((temp - temp0)/(temp - a4)))
    elif temp < temp0 - 23:
        a3 = a3i
        a4 = a4i
        es = a1 * np.exp(a3 * ((temp - temp0)/(temp - a4)))
    else:
        esw = a1 * np.exp(a3w * ((temp - temp0)/(temp - a4w)))
        esi = a1 * np.exp(a3i * ((temp - temp0)/(temp - a4i)))
        es = esi + ((esw - esi)*(((temp - (temp0 - 23))/23)**2))

    # get saturation specific humidity value
    epsilon = 0.622
    qs = (epsilon * es) / (pres - ((1 - epsilon)*es))
    return qs

In [54]:
def calc_qs(temp, pres):
    time_range = len(temp)
    pres_range = len(pres)
    qs = np.empty((time_range, pres_range))
    for time in range(time_range):
        for pres in range(len(pres)):
            qs[time, pres] = get_qs(temp[time, pres], pres[pres])
    return qs

In [49]:
t1 = t1.load()
pres = pres.load()
w1 = w1.load()
p1 = p1.load()
# q1 = q1.load()

In [59]:
# qs = calc_qs(t1.sel(lat=19, lon=72), pres)
qs = calc_qs(t1.to_numpy(), pres.to_numpy())

TypeError: 'int' object is not subscriptable

In [247]:
from scipy import integrate

In [248]:
# vert integral function (Simpson's method)
def vert_integ(x, y):
    int = integrate.simpson(y, x, even='avg')

    return int

# finite differnce methods to find derivative
def centered_diff(arr):
    arr_diff = np.empty(len(arr) - 2)
    for i in range((len(arr) - 2)):
        arr_diff[i] = arr[i+2] - arr[i]
    return arr_diff

def forward_diff(arr):
    arr_diff = np.diff(arr)
    return arr_diff

def backward_diff(arr):
    arr_diff = -(np.diff(arr[::-1])[::-1])
    return arr_diff

In [260]:
p_cdiff = centered_diff(pres)
p_fdiff = forward_diff(pres)
p_bdiff = backward_diff(pres)
#
qs_cdiff = centered_diff(qs)/(p_cdiff)
qs_fdiff = forward_diff(qs)/(p_fdiff)
qs_bdiff = backward_diff(qs)/(p_bdiff)

qs_diff = np.insert(qs_cdiff, 0, qs_fdiff[0])
qs_diff = np.append(qs_diff, qs_bdiff[-1])
# p_diff = np.insert(p_cdiff, 0, p_fdiff[0])
# p_diff = np.append(p_diff, p_bdiff[-1])

pe = (-100/9.806) * vert_integ(pres, (w1.sel(lat=19, lon=72))*qs_diff)

In [261]:
p1.sel(lat=19, lon=72)

<xarray.DataArray 'precipCal' ()>
array(44.30081, dtype=float32)
Coordinates:
    time     datetime64[ns] 2000-06-02T01:00:00
    lon      float64 72.0
    lat      float64 19.0

In [262]:
pe*1000

83.04520492621101

In [256]:
err = ((pe*1000 - p1.sel(lat=19, lon=72)) / p1.sel(lat=19, lon=72)) * 100
err.values

array(87.45752417)

In [176]:
# t2m = t2m.chunk(dict(time=5000, lat=40, lon=40))
# d2m = d2m.chunk(dict(time=5000, lat=40, lon=40))

In [47]:
# ds_comb = xr.merge([precip, t2m, d2m])

In [48]:
# ds_comb